# Machine Learning Pipeline - Feature Engineering

1. Data Analysis
2. **Feature Engineering**
3. Feature Selection
4. Model Training

In [72]:
# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt

# for the yeo-johnson transformation
import scipy.stats as stats

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler
# for the yeo-johnson transformation
import scipy.stats as stats

# to save the trained scaler class
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [73]:
# load dataset
df = pd.read_pickle("prueba_tecnica_2023.pkl", compression="bz2")
# dimensionality
print(df.shape)
# check first rows of the dataframe
df.head()

(30000, 65)


,created,order_amount,downpayment_amount,principal_amount,num_instalments_initial,test_ab_version,customer,scheduled_first_payment_date,pm_card_expiration_year,pm_card_expiration_monht,payment_method_uuid,pm_bincode,pm_card_bank,pm_card_country_code,pm_card_level,pm_card_type,customer_created,postcode,region_cp,province_cp,locality_cp,pm_is_prepaid,pm_has_bank,pm_is_national,target,ba_postcode,cust_phone_country,cust_phone_prefix,checkout_user_agent_browser_family,checkout_user_agent_browser_version,checkout_user_agent_device_family,checkout_user_agent_device_brand,checkout_user_agent_device_model,checkout_user_agent_os_family,checkout_user_agent_os_version,checkout_user_agent_is_pc,checkout_user_agent_is_mobile,checkout_user_agent_is_tablet,checkout_user_agent_is_touch_capable,ip_addresses_times,n_ips,n_distinct_ips,item_ids,total_price,item_prices,device_screen,device_browser_version,device_browser_language,device_browser_type,device_cookies_enabled,real_ip_parent_organization,real_ip_city,real_ip_region,real_ip_isp,providerZ_score,ip_userType,ip_reputation,log_providerA_score,ip_address.traits.user_type,providerY_score,payment_method_card_type,bank_countrycode,creditcard_level,creditcard_type,birthday_year
0,2020-09-04,208.00,9.904286,198.095714,20.0,B,cc888cc43ff54eb98cb4f41373f6f606,2020-10-02,NaN,NaN,b3420c23777a4d4cbf9803a2dc0b1125,NaN,NaN,NaN,NaN,NaN,2021-02-06,21130,Andalucia,Huelva,Palos de la Frontera,False,True,False,0.0,21130,ES,34.0,Chrome,84.0.4147,other,NaN,NaN,windows,10,1.0,0.0,0.0,0.0,"['2020-09-03T17:58:53.472478+00:00', '2020-09-...",4,1,b56ae8abb68643819eb442d4cee2feb8,198.9966,164.46,768X1366,84.0.4147.135,"ES-ES,ES;Q=0.9,EN;Q=0.8",CHROME,1.0,YA.COM INTERNET FACTORY,CASARICHE,SEVILLA,ORANGE ESPAGNE SA,122.0,wifi,1.0,-2.302585,cellular,4.0,NaN,NaN,NaN,NaN,2000
1,2020-10-02,34.19,2.180000,32.010000,16.0,B,996046636e2145b8958813e7c9645bb8,2020-11-05,2022.0,12.0,8a5357618002432b89d30be45521ac0b,485720.0,"BANKIA, S.A.",ES,CLASSIC,DEBIT,2021-01-07,28936,Madrid,Madrid,Móstoles,False,True,True,0.0,29004,ES,34.0,Chrome Mobile,85.0.4183,xiaomi redmi note 9s,XiaoMi,Redmi Note 9S,android,10,0.0,1.0,0.0,1.0,"['2020-10-01T20:57:30.427133+00:00', '2020-10-...",5,1,"079096ad74f3415e9a8323a972a26d23,0819966545404...",30.2016,"16.49,8.47",833X375,85.0.4183.127,"ES-ES,ES;Q=0.9",CHROME,0.0,DIGI SPAIN RESIDENTIAL,LOS CARMENES,MADRID,DIGI SPAIN TELECOM S.L.U.,65.0,wifi,1.0,-2.302585,residential,2.0,visa,ES,CLASSIC,DEBIT,1982
2,2020-12-11,500.77,22.480000,478.290000,24.0,B,8bf9a03537674050a60073279152b8a2,2021-01-10,2024.0,12.0,9aed0843c6a3413db3a61dcd8c6958ec,476664.0,"CAIXABANK, S.A.",ES,ELECTRON,DEBIT,2020-12-11,38780,Canarias,Santa Cruz de Tenerife,Tijarafe,False,True,True,0.0,38780,ES,34.0,Opera,71.0.3770,other,NaN,NaN,windows,10,1.0,0.0,0.0,0.0,"['2020-12-10T15:08:55.317154+00:00', '2020-12-...",7,1,83e47125c4fd484692418b4e49fec4c0,486.2722,454.46,1024X1280,71.0.3770.441,"ES-ES,ES;Q=0.9",OPR,1.0,YA.COM INTERNET FACTORY,LLANOS DE ARIDANE,SANTA CRUZ DE TENERIFE,ORANGE ESPAGNE SA,78.0,wired,1.0,-1.347074,residential,4.0,visa,ES,ELECTRON,DEBIT,1994
3,2020-11-30,184.95,11.750000,173.200000,16.0,B,3b905c533e034332a61f83933fa11f80,2020-12-28,NaN,NaN,d6d068dda51c4e7f96812b8957055fea,NaN,NaN,NaN,NaN,NaN,2020-11-30,16740,Castilla - La Mancha,Cuenca,"Almarcha, La",False,True,False,0.0,16740,ES,34.0,Chrome,86.0.4240,other,NaN,NaN,windows,10,1.0,0.0,0.0,0.0,"['2020-11-30T08:59:17.040043+00:00', '2020-11-...",5,1,7b026c2781024accae23864ea14e59c7,178.9953,147.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.0,wifi,1.0,-2.302585,residential,4.0,NaN,NaN,NaN,NaN,1981
4,2020-09-18,1201.24,53.910000,1147.330000,24.0,B,2d7ea31da4864854997abb5fcd623a06,2020-10-17,2023.0,3.0,5113fc67a5704db59d66ad28fe643d19,491767.0,"ABANCA CORPORACION BANCARIA, S.A.",ES,ELECTRON,DEBIT,2020-09-18,32635,Galicia,Ourense,Xinzo de Limia,False,True,True,0.0,32430,ES,34.0,Chrome Mobile,85.0.4183,samsung sm-g975f,Samsung,SM-G975F,android,10,0.0,1.0,0.0,1.0,"['2020-09-17T16:

In [74]:
# Fixing item_prices to get number of items
df['n_item'] = df['item_prices'].apply(lambda x : len(x.split(',')))

In [76]:
# drop id columns with high cardinality and useless columns
cols_to_drop = ['created','scheduled_first_payment_date','customer_created','pm_card_expiration_year','pm_card_expiration_monht','customer','payment_method_uuid','ip_addresses_times','item_ids','item_prices']
df.drop(cols_to_drop, axis =1, inplace = True)

In [77]:
df.shape

(30000, 56)

In [78]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('target', axis=1), # predictive variables
    df['target'], # target
    test_size=0.2, # portion of dataset to allocate to test set
    random_state=0, # we are setting the seed here
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((24000, 55), (6000, 55), (24000,), (6000,))

*****
### Feature Engineering

In the following cells, we will engineer the variables so that we tackle:
* Missing values
* Temporal variables
* Non-Gaussian distributed variables
* Categorical variables: remove rare labels
* Categorical variables: convert strings to numbers
* Put the variables in a similar scale

*****
#### Missing values Categorical variables

We will replace missing values with the string "missing" in those variables with a lot of missing data.

Alternatively, we will replace missing data with the most frequent category in those variables that contain fewer observations without values.

In [79]:
# getting lit of categorical variables
cat_vars = [var for var in X_train.columns if X_train[var].dtype == 'O']
# cast all variables as categorical
X_train[cat_vars] = X_train[cat_vars].astype('O')
X_test[cat_vars] = X_test[cat_vars].astype('O')

In [80]:
# categorical variables with missing information
cat_vars_with_na = [var for var in cat_vars if X_train[var].isna().sum() > 0]
# print percentage of missing values per variable
X_train[cat_vars_with_na ].isnull().mean().sort_values(ascending=False)

checkout_user_agent_device_brand       0.450583
checkout_user_agent_device_model       0.450583
creditcard_level                       0.211292
creditcard_type                        0.159750
bank_countrycode                       0.159750
payment_method_card_type               0.159750
pm_card_bank                           0.136958
pm_card_level                          0.125417
pm_card_country_code                   0.123000
pm_card_type                           0.123000
checkout_user_agent_os_version         0.043583
real_ip_parent_organization            0.043167
test_ab_version                        0.032542
device_browser_language                0.020042
checkout_user_agent_browser_version    0.018958
real_ip_isp                            0.017292
real_ip_region                         0.016917
device_screen                          0.016750
device_browser_version                 0.016750
device_browser_type                    0.016750
real_ip_city                           0

In [82]:
# variables to impute with the string missing
with_string_missing = [var for var in cat_vars_with_na if X_train[var].isnull().mean() > 0.1]
# variables to impute with the most frequent category
with_frequent_category = [var for var in cat_vars_with_na if X_train[var].isnull().mean() < 0.1]
# replace missing values with new label: "Missing"
X_train[with_string_missing] = X_train[with_string_missing].fillna('Missing')
X_test[with_string_missing] = X_test[with_string_missing].fillna('Missing')

#### Categorical variable missing information replacement with most frequent

In [83]:
for var in with_frequent_category:
    # there can be more than 1 mode in a variable
    # we take the first one with [0]
    mode = X_train[var].mode()[0]
    print(var, mode)
    X_train[var].fillna(mode, inplace=True)
    X_test[var].fillna(mode, inplace=True)

test_ab_version B
postcode 28007
region_cp Andalucia
province_cp Madrid
locality_cp Madrid
ba_postcode 28007
cust_phone_country ES
checkout_user_agent_browser_family Chrome
checkout_user_agent_browser_version 86.0.4240
checkout_user_agent_device_family other
checkout_user_agent_os_family windows
checkout_user_agent_os_version 10
device_screen 1080X1920
device_browser_version 604.1
device_browser_language ES-ES,ES;Q=0.9
device_browser_type CHROME
real_ip_parent_organization TELEFONICA DE ESPANA SAU
real_ip_city MADRID
real_ip_region MADRID
real_ip_isp ORANGE ESPAGNE SA
ip_userType wifi
ip_address.traits.user_type residential


In [84]:
# check that we have no missing information in the engineered variables
X_train[cat_vars_with_na].isnull().sum()

test_ab_version                        0
pm_card_bank                           0
pm_card_country_code                   0
pm_card_level                          0
pm_card_type                           0
postcode                               0
region_cp                              0
province_cp                            0
locality_cp                            0
ba_postcode                            0
cust_phone_country                     0
checkout_user_agent_browser_family     0
checkout_user_agent_browser_version    0
checkout_user_agent_device_family      0
checkout_user_agent_device_brand       0
checkout_user_agent_device_model       0
checkout_user_agent_os_family          0
checkout_user_agent_os_version         0
device_screen                          0
device_browser_version                 0
device_browser_language                0
device_browser_type                    0
real_ip_parent_organization            0
real_ip_city                           0
real_ip_region  

In [85]:
# check that test set does not contain null values in the engineered variables
[var for var in cat_vars_with_na if X_test[var].isnull().sum() > 0]

[]

### Numerical variables

To engineer missing values in numerical variables, we will:
* add a binary missing indicator variable
* and then replace the missing values in the original variable with the mean

In [87]:
# now let's identify the numerical variables
num_vars = [var for var in X_train.columns if var not in cat_vars]
# number of numerical variables
len(num_vars)

23

In [88]:
# make a list with the numerical variables that contain missing values
vars_with_na = [var for var in num_vars if X_train[var].isnull().sum() > 0]

# print percentage of missing values per variable
X_train[vars_with_na].isnull().mean()

pm_bincode                              0.123000
cust_phone_prefix                       0.000542
checkout_user_agent_is_pc               0.001292
checkout_user_agent_is_mobile           0.001292
checkout_user_agent_is_tablet           0.001292
checkout_user_agent_is_touch_capable    0.001292
device_cookies_enabled                  0.016750
providerZ_score                         0.003542
ip_reputation                           0.003542
log_providerA_score                     0.003167
providerY_score                         0.007542
dtype: float64

In [89]:
# replace missing values as we described above
for var in vars_with_na:
    # calculate the mean using the train set
    mean_val = X_train[var].mean()
    print(var, mean_val)
    # add binary missing indicator (in train and test)
    X_train[var + '_na'] = np.where(X_train[var].isnull(), 1, 0)
    X_test[var + '_na'] = np.where(X_test[var].isnull(), 1, 0)
    # replace missing values by the mean
    # (in train and test)
    X_train[var].fillna(mean_val, inplace=True)
    X_test[var].fillna(mean_val, inplace=True)

# check that we have no more missing values in the engineered variables
X_train[vars_with_na].isnull().sum()

pm_bincode 488546.8225959711
cust_phone_prefix 34.026389294201024
checkout_user_agent_is_pc 0.48153865409487256
checkout_user_agent_is_mobile 0.5084066919771372
checkout_user_agent_is_tablet 0.013267136718261087
checkout_user_agent_is_touch_capable 0.518336184237974
device_cookies_enabled 0.8607085346215781
providerZ_score 231.96420656491742
ip_reputation 0.9982019652937487
log_providerA_score -1.5377774635712165
providerY_score 3.7271505940635627


pm_bincode                              0
cust_phone_prefix                       0
checkout_user_agent_is_pc               0
checkout_user_agent_is_mobile           0
checkout_user_agent_is_tablet           0
checkout_user_agent_is_touch_capable    0
device_cookies_enabled                  0
providerZ_score                         0
ip_reputation                           0
log_providerA_score                     0
providerY_score                         0
dtype: int64

In [90]:
# check that test set does not contain null values in the engineered variables
[var for var in vars_with_na if X_test[var].isnull().sum() > 0]

[]

In [91]:
# check the binary missing indicator variables
X_train[['pm_bincode_na', 'cust_phone_prefix_na', 'checkout_user_agent_is_pc_na']].head()

,pm_bincode_na,cust_phone_prefix_na,checkout_user_agent_is_pc_na
3225,0,0,0
11815,0,0,0
7338,0,0,0
14980,0,0,0
27167,1,0,0


#### Numerical variable transformation Logarithmic transformation

In the previous notebook, we observed that the numerical variables are not normally distributed.

We will transform with the logarightm the positive numerical variables in order to get a more Gaussian-like distribution.

In [92]:
for var in ['order_amount', 'downpayment_amount', 'principal_amount','total_price']:
    X_train[var] = np.log(X_train[var])
    X_test[var] = np.log(X_test[var])

In [93]:
# check that test set does not contain null values in the engineered variables
[var for var in ['order_amount', 'downpayment_amount', 'principal_amount','total_price'] if X_test[var].isnull().sum() > 0]

[]

In [94]:
# same for train set
[var for var in ['order_amount', 'downpayment_amount', 'principal_amount','total_price'] if X_train[var].isnull().sum() > 0]

[]

### Removing Rare Labels

For the remaining categorical variables, we will group those categories that are present in less than 1% of the observations. That is, all values of categorical variables that are shared by less than 1% of transactions, well be replaced by the string "Rare".


In [95]:
def find_frequent_labels(df, var, rare_perc):
    # function finds the labels that are shared by more than
    # a certain % of the transactions in the dataset
    df = df.copy()
    tmp = df.groupby(var)[var].count() / len(df)
    return tmp[tmp > rare_perc].index

In [96]:
for var in cat_vars:
    # find the frequent categories
    frequent_ls = find_frequent_labels(X_train, var, 0.01)
    print(var, frequent_ls)
    print()
    # replace rare categories by the string "Rare"
    X_train[var] = np.where(X_train[var].isin(
        frequent_ls), X_train[var], 'Rare')
    X_test[var] = np.where(X_test[var].isin(
        frequent_ls), X_test[var], 'Rare')

test_ab_version Index(['A', 'B'], dtype='object', name='test_ab_version')

pm_card_bank Index(['ABANCA CORPORACION BANCARIA, S.A.',
       'BANCO BILBAO VIZCAYA ARGENTARIA S.A. (BBVA)',
       'BANCO COOPERATIVO ESPANOL, S.A.',
       'BANCO DE CREDITO COOPERATIVO (GRUPO CAJAMAR)', 'BANCO SABADELL, S.A.',
       'BANCO SANTANDER, S.A.', 'BANKIA, S.A.', 'CAIXABANK, S.A.',
       'IBERCAJA BANCO, S.A.', 'ING DIRECT N.V. SUCURSAL EN ESPANA',
       'LIBERBANK, S.A.', 'Missing', 'OPEN BANK SANTANDER CONSUMER',
       'SERVICIOS FINANCIEROS CARREFOUR, ESTABLECIMIENTO',
       'UNICAJA BANCO, S.A.'],
      dtype='object', name='pm_card_bank')

pm_card_country_code Index(['ES', 'Missing'], dtype='object', name='pm_card_country_code')

pm_card_level Index(['BUSINESS', 'CLASSIC', 'ELECTRON', 'ELECTRONIC', 'GOLD', 'Missing',
       'PLATINUM', 'PREPAID', 'STANDARD'],
      dtype='object', name='pm_card_level')

pm_card_type Index(['CREDIT', 'DEBIT', 'Missing'], dtype='object', name='pm_card_type

### Encoding of categorical variables

Next, we need to transform the strings of the categorical variables into numbers.
We will do it so that we capture the monotonic relationship between the label and the target.

In [97]:
def replace_categories(train, test, y_train, var, target):
    # Concatenate the feature variable with the target (y_train)
    tmp = pd.concat([train[[var]], y_train.rename(target)], axis=1)
    
    # Calculate the proportion of the positive class (1.0) for each category
    ordered_labels = tmp.groupby([var])[target].mean().sort_values().index

    # Create a dictionary to map each category to an ordinal integer value
    ordinal_label = {k: i for i, k in enumerate(ordered_labels, 0)}
    
    print(var, ordinal_label)
    print()

    # Replace the categorical values in both train and test sets with their corresponding ordinal values
    train[var] = train[var].map(ordinal_label)
    test[var] = test[var].map(ordinal_label)

In [98]:
for var in cat_vars:
    replace_categories(X_train, X_test, y_train, var, 'target')

test_ab_version {'B': 0, 'A': 1}

pm_card_bank {'UNICAJA BANCO, S.A.': 0, 'IBERCAJA BANCO, S.A.': 1, 'BANCO COOPERATIVO ESPANOL, S.A.': 2, 'BANCO DE CREDITO COOPERATIVO (GRUPO CAJAMAR)': 3, 'SERVICIOS FINANCIEROS CARREFOUR, ESTABLECIMIENTO': 4, 'LIBERBANK, S.A.': 5, 'BANCO SABADELL, S.A.': 6, 'CAIXABANK, S.A.': 7, 'ING DIRECT N.V. SUCURSAL EN ESPANA': 8, 'BANCO SANTANDER, S.A.': 9, 'BANKIA, S.A.': 10, 'Rare': 11, 'Missing': 12, 'BANCO BILBAO VIZCAYA ARGENTARIA S.A. (BBVA)': 13, 'ABANCA CORPORACION BANCARIA, S.A.': 14, 'OPEN BANK SANTANDER CONSUMER': 15}

pm_card_country_code {'Missing': 0, 'ES': 1, 'Rare': 2}

pm_card_level {'Missing': 0, 'ELECTRON': 1, 'PLATINUM': 2, 'ELECTRONIC': 3, 'STANDARD': 4, 'BUSINESS': 5, 'GOLD': 6, 'CLASSIC': 7, 'Rare': 8, 'PREPAID': 9}

pm_card_type {'Missing': 0, 'CREDIT': 1, 'DEBIT': 2}

postcode {'Rare': 0, '28007': 1}

region_cp {'Castilla - Leon': 0, 'Pais Vasco': 1, 'Castilla - La Mancha': 2, 'Canarias': 3, 'Aragon': 4, 'Baleares': 5, 'Rare': 6, 'Murci

In [99]:
# check absence of na in the train set
[var for var in X_train.columns if X_train[var].isnull().sum() > 0]

[]

In [100]:
# check absence of na in the test set
[var for var in X_test.columns if X_test[var].isnull().sum() > 0]

[]

### Feature Scaling
For use in linear models, features need to be either scaled. We will scale features to the minimum and maximum values:


In [101]:
# create scaler
scaler = MinMaxScaler()
#  fit  the scaler to the train set
scaler.fit(X_train) 
# transform the train and test set
# sklearn returns numpy arrays, so we wrap the
# array with a pandas dataframe
X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns=X_train.columns
)
X_test = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_train.columns
)

In [102]:
X_train.head()

,order_amount,downpayment_amount,principal_amount,num_instalments_initial,test_ab_version,pm_bincode,pm_card_bank,pm_card_country_code,pm_card_level,pm_card_type,postcode,region_cp,province_cp,locality_cp,pm_is_prepaid,pm_has_bank,pm_is_national,ba_postcode,cust_phone_country,cust_phone_prefix,checkout_user_agent_browser_family,checkout_user_agent_browser_version,checkout_user_agent_device_family,checkout_user_agent_device_brand,checkout_user_agent_device_model,checkout_user_agent_os_family,checkout_user_agent_os_version,checkout_user_agent_is_pc,checkout_user_agent_is_mobile,checkout_user_agent_is_tablet,checkout_user_agent_is_touch_capable,n_ips,n_distinct_ips,total_price,device_screen,device_browser_version,device_browser_language,device_browser_type,device_cookies_enabled,real_ip_parent_organization,real_ip_city,real_ip_region,real_ip_isp,providerZ_score,ip_userType,ip_reputation,log_providerA_score,ip_address.traits.user_type,providerY_score,payment_method_card_type,bank_countrycode,creditcard_level,creditcard_type,birthday_year,n_item,pm_bincode_na,cust_phone_prefix_na,checkout_user_agent_is_pc_na,checkout_user_agent_is_mobile_na,checkout_user_agent_is_tablet_na,checkout_user_agent_is_touch_capable_na,device_cookies_enabled_na,providerZ_score_na,ip_reputation_na,log_providerA_score_na,providerY_score_na
0,0.931634,0.825557,0.927385,0.379310,0.0,0.549267,0.466667,0.5,0.111111,1.0,0.0,0.928571,0.740741,0.285714,0.0,1.0,1.0,1.0,1.0,0.0,0.636364,0.785714,0.4,0.000000,0.4,0.5,0.75,0.0,1.0,0.0,1.0,0.102041,0.0,0.898705,0.80,0.666667,0.500,0.142857,1.0,0.55,0.625,0.714286,0.888889,0.000000,0.666667,1.0,0.000000,0.0,0.4,0.666667,0.5,0.285714,1.0,0.708333,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.699385,0.574441,0.699721,0.551724,0.0,0.631802,0.866667,0.5,0.111111,1.0,0.0,0.857143,0.296296,0.285714,0.0,1.0,1.0,0.0,1.0,0.0,0.181818,0.928571,0.6,0.571429,0.6,0.0,0.75,1.0,0.0,0.0,0.0,0.040816,0.0,0.546072,0.25,0.962963,0.500,0.857143,1.0,0.55,0.625,0.047619,0.444444,0.110772,0.666667,1.0,0.000000,0.0,0.6,0.666667,0.5,0.285714,1.0,0.902778,0.040816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.833097,0.659888,0.834303,0.965517,0.0,0.549273,0.466667,0.5,0.111111,1.0,0.0,0.857143,0.481481,0.285714,0.0,1.0,1.0,0.0,1.0,0.0,0.636364,0.214286,0.4,0.428571,0.4,0.5,0.75,0.0,1.0,0.0,1.0,0.081633,0.0,0.751263,0.80,0.629630,0.375,0.142857,1.0,0.55,0.625,0.380952,0.333333,0.496951,1.000000,1.0,0.652363,0.5,0.6,0.666667,0.5,0.285714,1.0,0.472222,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.713044,0.550666,0.715117,0.965517,0.0,0.822985,0.733333,0.5,0.888889,1.0,0.0,0.857143,0.851852,0.285714,1.0,1.0,1.0,0.0,1.0,0.0,0.636364,0.214286,0.4,0.714286,0.4,0.5,0.75,0.0,1.0,0.0,1.0,0.040816,0.0,0.562842,0.80,0.629630,0.500,0.142857,1.0,0.30,0.625,0.142857,0.333333,0.000000,0.666667,1.0,0.000000,0.0,0.4,1.000000,0.5,0.571429,1.0,0.513889,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.973092,0.871215,0.967814,0.344828,0.0,0.613849,0.800000,0.0,0.000000,0.0,0.0,0.428571,0.259259,0.285714,0.0,1.0,0.0,0.0,1.0,0.0,0.636364,0.142857,0.4,0.428571,0.4,0.5,0.75,0.0,1.0,0.0,1.0,0.183673,0.0,0.960739,0.15,0.962963,0.125,0.142857,1.0,0.55,0.625,0.666667,0.666667,0.218496,0.666667,1.0,0.630310,0.0,0.4,0.333333,0.0,0.142857,0.0,0.847222,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [103]:
# let's now save the train and test sets for the next notebook!
X_train.to_csv('xtrain_scaled.csv', index=False)
X_test.to_csv('xtest_scaled.csv', index=False)

y_train.to_csv('ytrain.csv', index=False)
y_test.to_csv('ytest.csv', index=False)

In [104]:
# now let's save the scaler
joblib.dump(scaler, 'minmax_scaler.joblib')

['minmax_scaler.joblib']